 # Manipuler les observables




Dans le formalisme de la mécanique quantique, une opération de mesure (c'est-à-dire obtenir la valeur ou un intervalle de valeurs d'un paramètre physique, ou plus généralement une information sur un système physique) est représentée par ce qu'il est convenu d'appeler une observable.

Une observable est formalisée mathématiquement par un opérateur agissant sur les vecteurs d'un espace de Hilbert $\mathcal{H}$ (chaque état quantique étant représenté par un vecteur dans cet espace).

Définissons notre observable :

In [2]:
from qat.core import Observable, Term
nbqbits = 5
one_count = Observable(nbqbits)

Il est possible d'ajouter des termes de Pauli à l'observable et ainsi de l'écrire comme une somme d'opérateurs de Pauli :

In [5]:
# The sigma Z terms:
for i in range(nbqbits):
    one_count.add_term(Term(-0.5, "Z", [i]))
# And the constant term:
one_count.constant_coeff += nbqbits/2

print(one_count)


7.5 * I^5 +
-1.5 * (Z|[0]) +
-1.5 * (Z|[1]) +
-1.5 * (Z|[2]) +
-1.5 * (Z|[3]) +
-1.5 * (Z|[4])


Utilisons notre observable sur un circuit :

In [6]:
from qat.lang.AQASM import Program, X, CNOT, RX

prog_2_ones = Program()
qbits = prog_2_ones.qalloc(nbqbits)
prog_2_ones.apply(X, qbits[0])
prog_2_ones.apply(CNOT, qbits[0], qbits[2])
circ_2_ones = prog_2_ones.to_circ()

from qat.qpus import PyLinalg
from qat.plugins import ObservableSplitter

qpu = ObservableSplitter() | PyLinalg()
job = circ_2_ones.to_job("OBS", observable=one_count, nbshots=30)
print("Number of ones:", qpu.submit(job).value)

Number of ones: 6.0
